# Preprocessing the TCGA PDAC dataset

CK 2022

### What this does:
- Loads in TCGA PDAC dataset
- Adds module scores to the data for
    - Prognostic PDAC signatures (e.g. Moffit et al Classical/Basal scores)
    - cNMF modules from validation experiments
    - MsigDB genesets related to the ligands in validation experiments

In [ ]:
library(Seurat)
library(tidyverse)
library(viridis)
library(stringr)
library(survival)
library(survminer)

Loading in the TCGA data into a Seurat object

In [ ]:
# Metadata
metadata = read.table("gdc_sample_sheet.2022-04-22.tsv",sep="\t")
colnames(metadata) = metadata[1,]
metadata = metadata[-1,]
rownames(metadata) = metadata$`Sample ID`

# Count files
count_files = Sys.glob("data/*.tsv")
for(i in 1:length(count_files)){
  counts = read.table(count_files[i],skip=6)
  colnames(counts) <- c("gene_id","gene_name","gene_type","unstranded","stranded_first","stranded_second","tpm_unstranded","fkpm_unstranded","fpkm_uq_unstranded")
  counts = counts %>% filter()
  
  sample_id = metadata$`Sample ID`[metadata$`File Name` == strsplit(count_files[i],"/")[[1]][2]]
  
  if(i == 1){
    counts_by_patient= data.frame(gene = counts$gene_name)
    counts_by_patient[sample_id] = counts$unstranded
  }else{
    temp = counts[,c("gene_name","unstranded")]
    counts_by_patient = cbind(counts_by_patient,temp$unstranded)
    colnames(counts_by_patient)[i+1] = sample_id
  }
}
print(dim(counts_by_patient))
write.table(counts_by_patient,"tcga_paad_unstranded_counts_by_sample.txt")

# Depulicating 
multiple_genes = unique(counts_by_patient$gene[duplicated(counts_by_patient$gene)])
duplicated_data = counts_by_patient %>% filter(gene %in% multiple_genes)
counts_by_patient = counts_by_patient %>% filter(!(gene %in% multiple_genes))

for(i in 1:length(multiple_genes)){
  gene_sums = colSums(duplicated_data %>% filter(gene==multiple_genes[i]) %>% select(-gene))
  if(i ==1){
    new_data = t(data.frame(gene_sums))
  }else{
    new_data = rbind(new_data,t(data.frame(gene_sums)))
  }
}
rownames(new_data) = multiple_genes

updated_counts = rbind(counts_by_patient[,-1],new_data)
updated_counts = cbind(c(counts_by_patient$gene,multiple_genes),updated_counts)
colnames(updated_counts)[1] = "gene"

# Making a Seurat object
temp = updated_counts
rownames(temp) = updated_counts$gene
temp = temp[,-1]
tcga_paad = CreateSeuratObject(counts=temp)

# adding in metadata
tcga_paad_sample_ids = rownames(tcga_paad@meta.data)
tcga_paad$sample_id = tcga_paad_sample_ids
metadata$sample_id = gsub("-",".",metadata$`Sample ID`)
tcga_paad@meta.data = tcga_paad@meta.data %>% left_join(metadata,by="sample_id")
rownames(tcga_paad@meta.data) = tcga_paad_sample_ids
tcga_paad@meta.data

# normalizing the data
tcga_paad = NormalizeData(tcga_paad)

Adding in prognositic gene signature module scores

In [ ]:
mClassical = read.table("../../BEM/PDAC_cs_analysis/1_Data/marker_genes/PDAC_ref_genes/Moffitt-classical25.txt")[,1]
mBasal = read.table("../../BEM/PDAC_cs_analysis/1_Data/marker_genes/PDAC_ref_genes/Moffitt-basal25.txt")[,1]
EMT_up = read.table("../../BEM/PDAC_cs_analysis/1_Data/marker_genes/PDAC_ref_genes/EMT_up.txt")[,1]
sc_org_spec = read.table("../../BEM/PDAC_cs_analysis/1_Data/marker_genes/PDAC_ref_genes/scOrg_Spec.txt")[,1]

scBasal = read.table("../../BEM/PDAC_cs_analysis/1_Data/marker_genes/PDAC_bioRxiv/scBasal.txt")[,1]
scClassical = read.table("../../BEM/PDAC_cs_analysis/1_Data/marker_genes/PDAC_bioRxiv/scClassical.txt")[,1]
scIT = read.table("../../BEM/PDAC_cs_analysis/1_Data/marker_genes/PDAC_bioRxiv/scIT.txt")[,1]

tcga_paad = AddModuleScore(tcga_paad,features = list(mClassical),name='mClassical')
tcga_paad = AddModuleScore(tcga_paad,features = list(mBasal),name='mBasal')
tcga_paad = AddModuleScore(tcga_paad,features = list(scClassical),name='scClassical')
tcga_paad = AddModuleScore(tcga_paad,features = list(scBasal),name='scBasal')
tcga_paad = AddModuleScore(tcga_paad,features = list(sc_org_spec),name='scOrganoidSpecific')
tcga_paad = AddModuleScore(tcga_paad,features = list(scIT),name='scIT')
tcga_paad = AddModuleScore(tcga_paad,features = list(EMT_up),name='EMT_up')

tcga_paad$mBC = tcga_paad$mClassical1 - tcga_paad$mBasal1
tcga_paad$scBC = tcga_paad$scClassical1 - tcga_paad$scBasal1

Adding in cNMF module scores

In [ ]:

old_modules = c('Module_2','Module_4','Module_5','Module_7',
                'Module_10','Module_12','Module_13','Module_14','Module_15','Module_17',
                'Module_18','Module_20','Module_23')
new_modules = c("low complexity 3","cell cycle G2M","low complexity 1","type I IFN",
                "NFKB","ribosomal","wound healing","mitochondrial","cell cycle S phase","IFNgamma",
                "classical","low complexity 2","TGFbeta")
module_key = data.frame(old=old_modules,new=new_modules)

cnmf_gene_lists = read.csv("../../BEM/PDAC_cs_analysis//4_Validation_combined_cNMF/figures/k_26_valid_d7_concat_knee_2_top_genes.csv")
cnmf_gene_lists = cnmf_gene_lists %>% select(module_key$old)
for(i in 1:ncol(cnmf_gene_lists)){
  gene_list = list(cnmf_gene_lists[,i])
  module_name = module_key$new[module_key$old==colnames(cnmf_gene_lists)[i]]
  tcga_paad = AddModuleScore(tcga_paad,features=gene_list,name=paste0(module_name,"_"))


Adding in select MsigDB signatures

In [ ]:
hallmark_tgfb =   c("ACVR1","APC","ARID4B","BCAR3","BMP2","BMPR1A","BMPR2","CDH1","CDK9","CDKN1C","CTNNB1","ENG","FKBP1A","FNTA","FURIN","HDAC1","HIPK2","ID1","ID2","ID3","IFNGR2","JUNB","KLF10","LEFTY2","LTBP2","MAP3K7","NCOR2","NOG","PMEPA1","PPM1A","PPP1CA","PPP1R15A","RAB31","RHOA","SERPINE1","SKI","SKIL","SLC20A1","SMAD1","SMAD3","SMAD6","SMAD7","SMURF1","SMURF2","SPTBN1","TGFB1","TGFBR1","TGIF1","THBS1","TJP1","TRIM33","UBE2D3","WWTR1","XIAP")
tcga_paad = AddModuleScore(tcga_paad,features=list(hallmark_tgfb),name="hallmark_tgfb")

hallmark_ifna =   c("ADAR","B2M","BATF2","BST2","C1S","CASP1","CASP8","CCRL2","CD47","CD74","CMPK2","CMTR1","CNP","CSF1","CXCL10","CXCL11","DDX60","DHX58","EIF2AK2","ELF1","EPSTI1","GBP2","GBP4","GMPR","HELZ2","HERC6","HLA-C","IFI27","IFI30","IFI35","IFI44","IFI44L","IFIH1","IFIT2","IFIT3","IFITM1","IFITM2","IFITM3","IL15","IL4R","IL7","IRF1","IRF2","IRF7","IRF9","ISG15","ISG20","LAMP3","LAP3","LGALS3BP","LPAR6","LY6E","MOV10","MVB12A","MX1","NCOA7","NMI","NUB1","OAS1","OASL","OGFR","PARP12","PARP14","PARP9","PLSCR1","PNPT1","PROCR","PSMA3","PSMB8","PSMB9","PSME1","PSME2","RIPK2","RNF31","RSAD2","RTP4","SAMD9","SAMD9L","SELL","SLC25A28","SP110","STAT2","TAP1","TDRD7","TENT5A","TMEM140","TRAFD1","TRIM14","TRIM21","TRIM25","TRIM26","TRIM5","TXNIP","UBA7","UBE2L6","USP18","WARS1")
tcga_paad = AddModuleScore(tcga_paad,features=list(hallmark_ifna),name="hallmark_ifna")

hallmark_ifng =   c("ADAR","APOL6","ARID5B","ARL4A","AUTS2","B2M","BANK1","BATF2","BPGM","BST2","BTG1","C1R","C1S","CASP1","CASP3","CASP4","CASP7","CASP8","CCL2","CCL5","CCL7","CD274","CD38","CD40","CD69","CD74","CD86","CDKN1A","CFB","CFH","CIITA","CMKLR1","CMPK2","CMTR1","CSF2RB","CXCL10","CXCL11","CXCL9","DDX58","DDX60","DHX58","EIF2AK2","EIF4E3","EPSTI1","FAS","FCGR1A","FGL2","FPR1","GBP4","GBP6","GCH1","GPR18","GZMA","HELZ2","HERC6","HIF1A","HLA-A","HLA-B","HLA-DMA","HLA-DQA1","HLA-DRB1","HLA-G","ICAM1","IDO1","IFI27","IFI30","IFI35","IFI44","IFI44L","IFIH1","IFIT1","IFIT2","IFIT3","IFITM2","IFITM3","IFNAR2","IL10RA","IL15","IL15RA","IL18BP","IL2RB","IL4R","IL6","IL7","IRF1","IRF2","IRF4","IRF5","IRF7","IRF8","IRF9","ISG15","ISG20","ISOC1","ITGB7","JAK2","KLRK1","LAP3","LATS2","LCP2","LGALS3BP","LY6E","LYSMD2","MARCHF1","METTL7B","MT2A","MTHFD2","MVP","MX1","MX2","MYD88","NAMPT","NCOA3","NFKB1","NFKBIA","NLRC5","NMI","NOD1","NUP93","OAS2","OAS3","OASL","OGFR","P2RY14","PARP12","PARP14","PDE4B","PELI1","PFKP","PIM1","PLA2G4A","PLSCR1","PML","PNP","PNPT1","PSMA2","PSMA3","PSMB10","PSMB2","PSMB8","PSMB9","PSME1","PSME2","PTGS2","PTPN1","PTPN2","PTPN6","RAPGEF6","RBCK1","RIPK1","RIPK2","RNF213","RNF31","RSAD2","RTP4","SAMD9L","SAMHD1","SECTM1","SELP","SERPING1","SLAMF7","SLC25A28","SOCS1","SOCS3","SOD2","SP110","SPPL2A","SRI","SSPN","ST3GAL5","ST8SIA4","STAT1","STAT2","STAT3","STAT4","TAP1","TAPBP","TDRD7","TNFAIP2","TNFAIP3","TNFAIP6","TNFSF10","TOR1B","TRAFD1","TRIM14","TRIM21","TRIM25","TRIM26","TXNIP","UBE2L6","UPP1","USP18","VAMP5","VAMP8","VCAM1","WARS1","XAF1","XCL1","ZBP1","ZNFX1")
tcga_paad = AddModuleScore(tcga_paad,features=list(hallmark_ifng),name="hallmark_ifng")

hallmark_emt =   c("ABI3BP","ACTA2","ADAM12","ANPEP","APLP1","AREG","BASP1","BDNF","BGN","BMP1","CADM1","CALD1","CALU","CAP2","CAPG","CCN1","CCN2","CD44","CD59","CDH11","CDH2","CDH6","COL11A1","COL12A1","COL16A1","COL1A1","COL1A2","COL3A1","COL4A1","COL4A2","COL5A1","COL5A2","COL5A3","COL6A2","COL6A3","COL7A1","COL8A2","COLGALT1","COMP","COPA","CRLF1","CTHRC1","CXCL1","CXCL12","CXCL6","CXCL8","DAB2","DCN","DKK1","DPYSL3","DST","ECM1","ECM2","EDIL3","EFEMP2","ELN","EMP3","ENO2","FAP","FAS","FBLN1","FBLN2","FBLN5","FBN1","FBN2","FERMT2","FGF2","FLNA","FMOD","FN1","FOXC2","FSTL1","FSTL3","FUCA1","FZD8","GADD45A","GADD45B","GAS1","GEM","GJA1","GLIPR1","GPC1","GPX7","GREM1","HTRA1","ID2","IGFBP2","IGFBP3","IGFBP4","IL15","IL32","IL6","INHBA","ITGA2","ITGA5","ITGAV","ITGB1","ITGB3","ITGB5","JUN","LAMA1","LAMA2","LAMA3","LAMC1","LAMC2","LGALS1","LOX","LOXL1","LOXL2","LRP1","LRRC15","LUM","MAGEE1","MATN2","MATN3","MCM7","MEST","MFAP5","MGP","MMP1","MMP14","MMP2","MMP3","MSX1","MXRA5","MYL9","MYLK","NID2","NNMT","NOTCH2","NT5E","NTM","OXTR","P3H1","PCOLCE","PCOLCE2","PDGFRB","PDLIM4","PFN2","PLAUR","PLOD1","PLOD2","PLOD3","PMEPA1","PMP22","POSTN","PPIB","PRRX1","PRSS2","PTHLH","PTX3","PVR","QSOX1","RGS4","RHOB","SAT1","SCG2","SDC1","SDC4","SERPINE1","SERPINE2","SERPINH1","SFRP1","SFRP4","SGCB","SGCD","SGCG","SLC6A8","SLIT2","SLIT3","SNAI2","SNTB1","SPARC","SPOCK1","SPP1","TAGLN","TFPI2","TGFB1","TGFBI","TGFBR3","TGM2","THBS1","THBS2","THY1","TIMP1","TIMP3","TNC","TNFAIP3","TNFRSF11B","TNFRSF12A","TPM1","TPM2","TPM4","VCAM1","VCAN","VEGFA","VEGFC","VIM","WIPF1","WNT5A")
tcga_paad = AddModuleScore(tcga_paad,features=list(hallmark_emt),name="hallmark_emt")

hallmark_tnfa_nfkb =   c("ABCA1","ACKR3","AREG","ATF3","ATP2B1","B4GALT1","B4GALT5","BCL2A1","BCL3","BCL6","BHLHE40","BIRC2","BIRC3","BMP2","BTG1","BTG2","BTG3","CCL2","CCL20","CCL4","CCL5","CCN1","CCND1","CCNL1","CCRL2","CD44","CD69","CD80","CD83","CDKN1A","CEBPB","CEBPD","CFLAR","CLCF1","CSF1","CSF2","CXCL1","CXCL10","CXCL11","CXCL2","CXCL3","CXCL6","DDX58","DENND5A","DNAJB4","DRAM1","DUSP1","DUSP2","DUSP4","DUSP5","EDN1","EFNA1","EGR1","EGR2","EGR3","EHD1","EIF1","ETS2","F2RL1","F3","FJX1","FOS","FOSB","FOSL1","FOSL2","FUT4","G0S2","GADD45A","GADD45B","GCH1","GEM","GFPT2","GPR183","HBEGF","HES1","ICAM1","ICOSLG","ID2","IER2","IER3","IER5","IFIH1","IFIT2","IFNGR2","IL12B","IL15RA","IL18","IL1A","IL1B","IL23A","IL6","IL6ST","IL7R","INHBA","IRF1","IRS2","JAG1","JUN","JUNB","KDM6B","KLF10","KLF2","KLF4","KLF6","KLF9","KYNU","LAMB3","LDLR","LIF","LITAF","MAFF","MAP2K3","MAP3K8","MARCKS","MCL1","MSC","MXD1","MYC","NAMPT","NFAT5","NFE2L2","NFIL3","NFKB1","NFKB2","NFKBIA","NFKBIE","NINJ1","NR4A1","NR4A2","NR4A3","OLR1","PANX1","PDE4B","PDLIM5","PER1","PFKFB3","PHLDA1","PHLDA2","PLAU","PLAUR","PLEK","PLK2","PLPP3","PMEPA1","PNRC1","PPP1R15A","PTGER4","PTGS2","PTPRE","PTX3","RCAN1","REL","RELA","RELB","RHOB","RIPK2","RNF19B","SAT1","SDC4","SERPINB2","SERPINB8","SERPINE1","SGK1","SIK1","SLC16A6","SLC2A3","SLC2A6","SMAD3","SNN","SOCS3","SOD2","SPHK1","SPSB1","SQSTM1","STAT5A","TANK","TAP1","TGIF1","TIPARP","TLR2","TNC","TNF","TNFAIP2","TNFAIP3","TNFAIP6","TNFAIP8","TNFRSF9","TNFSF9","TNIP1","TNIP2","TRAF1","TRIB1","TRIP10","TSC22D1","TUBB2A","VEGFA","YRDC","ZBTB10","ZC3H12A","ZFP36")
tcga_paad = AddModuleScore(tcga_paad,features=list(hallmark_tnfa_nfkb),name="hallmark_tnfa")

biocarta_il4_pathway = c('AKT1','GRB2','IL2RG','IL4','IL4R','IRS1','JAK1','JAK3','RPS6KB1','SHC1','STAT6')
tcga_paad = AddModuleScore(tcga_paad,features=list(biocarta_il4_pathway),name="biocarta_il4")
liu_il4_signaaling =   c("ALOX5","ANKRD33B","APPL2","ARPC5","ATPAF1","ATXN1","BATF","BCKDK","BCL6","BHLHE40","CCL4","CCL5","CCN2","CD69","CD79B","CD86","CFLAR","CISH","CLTC","CMTM6","CNTN6","CTSD","EBF1","EGR1","EIF4G1","EVI2A","FCER2","FGL2","GCSAM","GNAS","GNG2","GSR","HLA-DRB5","HLX","HOMER2","HOXC4","HUWE1","IL2RA","IL4R","IL6","IRF8","KLF13","KMO","LY75","MAL","MAP3K5","METTL14","MGLL","MOK","MRPS15","MTCL1","MTMR9","NCF2","NCOA3","NEK6","NFIL3","NFKBIZ","PDE7A","PEG10","PHF20L1","PKN2","PMAIP1","PPP2CA","PPP6C","QSOX1","RASGRP3","RGS6","RPL5","RPS2","SAMSN1","SEL1L3","SELL","SIAH2","SLA","SLC15A2","SLC39A8","SMARCA2","SOCS1","SPAG1","SPINT2","STAT4","STK4","TLR3","TLR7","TNFAIP3","TNFRSF11A","VCL","VEGFC","WT1","XBP1","ZBED2")
tcga_paad = AddModuleScore(tcga_paad,features=list(liu_il4_signaaling),name="liu_il4")

reactome_il4_signaling = c("AKT1","ALOX15","ALOX5","ANXA1","BATF","BCL2","BCL2L1","BCL6","BIRC5","CCL11","CCL2","CCL22","CCND1","CD36","CDKN1A","CEBPD","COL1A2","CXCL8","F13A1","FASLG","FCER2","FGF2","FN1","FOS","FOXO1","FOXO3","FSCN1","GATA3","HGF","HIF1A","HMOX1","HSP90AA1","HSP90B1","HSPA8","ICAM1","IGHE","IGHG1","IGHG4","IL10","IL12A","IL12B","IL13","IL13RA1","IL13RA2","IL17A","IL17F","IL18","IL1A","IL1B","IL23A","IL23R","IL2RG","IL4","IL4R","IL6","IL6R","IRF4","ITGAM","ITGAX","ITGB1","ITGB2","JAK1","JAK2","JAK3","JUNB","LAMA5","LBP","LCN2","LIF","MAOA","MCL1","MMP1","MMP2","MMP3","MMP9","MUC1","MYC","NANOG","NDN","NOS2","OPRD1","OPRM1","OSM","PIK3R1","PIM1","POMC","POU2F1","PTGS2","RHOU","RORA","RORC","S1PR1","SAA1","SOCS1","SOCS3","SOCS5","SOX2","STAT1","STAT3","STAT6","TGFB1","TIMP1","TNF","TNFRSF1B","TP53","TWIST1","TYK2","VCAM1","VEGFA","VIM","ZEB1")
tcga_paad = AddModuleScore(tcga_paad,features=list(reactome_il4_signaling),name="reactome_il4")

Writing csv file of the metadata

In [ ]:
write.csv(tcga_paad@meta.data,"tcga_paad_meta_data.csv")